
## Coursework 1
## Introduction to computer vision


#### Point 1

In [ ]:
!pip install opencv-python

In [ ]:
import cv2
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
import os

In [ ]:
img=cv2.imread('my_name.jpeg')

In [ ]:
def read_img(path):
    img=cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#### 1.B Transformations

**Rotated Images**

The first transformation to the image is the rotation. To achieve this, I have defined 3 methods

The first method is Interpolate, which is used when forward mapping is applied (you can set the mapping preference). Basically, this method applies the median kernel to avoid black pixels in the output image. 


This is a helper method, you don't need to interact with it.

In [ ]:
#This method applies the median filter
def ICV_interpolate(img):
    #first the height and weight of the kernel and the image are stored in variables
    w_kernel,h_kernel = 3, 3
    w_img,h_img = len(img),len(img[0])
    #A matrix with the same dimensions than the input image is created
    new_img = np.zeros_like(img)
    #Iteration starts from column and row 1 and finishes in size of the image - 1
    for i in range(1,w_img-1):
        for j in range(1,h_img-1):
            #Every pixel is replaced with the median value found in the 3x3 matrix around
            new_img[i,j,0] = np.median(img[i-1:i+2, j-1:j+2,0])
            new_img[i,j,1] = np.median(img[i-1:i+2, j-1:j+2,1])
            new_img[i,j,2] = np.median(img[i-1:i+2, j-1:j+2,2])
    return new_img

When rotating the image, some data may be lost because the length of the image is insufficient. The following method was developed to avoid this. Basically, it calculates the diagonal of the image and compares it to the length of the image. This might increase the execution time of the rotation method, but it ensures that all information is preserved.


This is a helper method, you don't need to interact with it.

In [ ]:
def ICV_add_borders(img):
    #The diagonal of the image is calculated (max possible length and width)
    c = np.sqrt(len(img[0])**2+len(img)**2)
    #If the width or height are smaller than the diagonal, the missing border is added to the image.
    borders = np.zeros_like(img[0:int((c-len(img))/2),:,:])
    img=np.vstack((borders,img,borders))
    bordersh = np.zeros_like(img[:,0:int((c-len(img[0]))/2),:])
    img=np.hstack((bordersh,img,bordersh))
    return img

Now, the rotation method is defined with the following parameters:

**Img**: the image to rotate in RGB or BGR format.


**Angle**: Angle to be rotated, must be in degrees.


**Inverse**: A flag to define the mapping method, by default inverse mapping is used.

This method basically multiplies each position of the input image by the rotation matrix defined in the class to obtain the output values and finally maps them using the selected mapping method.

In [ ]:
def ICV_rotate(img,angle,inverse=True):
    #adding borders to avoid losing pixels when transforming
    img=ICV_add_borders(img)
    #in case of inverse mapping it is necessary to take the negative angle
    if inverse:
        angle=-angle
    #Numpy works with the angle in radians, and the input is a value in degrees
    angle=np.deg2rad(angle)
    #Rotation matrix is defined and a matrix with the same dimensions than the image is created
    rot_matrix=[[np.cos(angle), -np.sin(angle)],
        [np.sin(angle), np.cos(angle)]]
    rotated=np.zeros_like(img)
    center=len(img)//2
    center_y=len(img[0])//2
    #The origin is defined in the center of the image
    for i in range(-center,center):
        for j in range(-center_y,center_y):
            #Each position is multiply by the rotation matrix defined to get the transformed position
            new_pos = np.dot(rot_matrix,np.array([[i],[j]]))
            new_pos_x=np.int_(new_pos[0])
            new_pos_y=np.int_(new_pos[1])
            
            if 0 <= new_pos_x+center and new_pos_x+center < len(img) and 0 <= new_pos_y+center_y and new_pos_y+center_y< len(img[0]):
                if inverse:
                    #In case of inverse mapping, searches for the calculated position into the input image and assigns its value to the output
                    rotated[i+center,j+center_y]=img[new_pos_x+center,new_pos_y+center_y]
                else:
                    #In case of forward mapping, assigns the value from the input to the calculated position in the output
                    rotated[new_pos_x+center,new_pos_y+center_y]=img[i+center,j+center_y]
    if inverse:
        return rotated
    else:
        return ICV_interpolate(rotated)


Next line can be used to execute the forward mapping rotation, i'm not executing it to avoid innecesary iterations. 



In [ ]:
#cv2.imwrite("fordward_median_rotated_30.jpg",ICV_rotate(img,30))

Rotation by using inverse mapping:

In [ ]:
cv2.imwrite("inverse_rot_30.jpg",ICV_rotate(img,30,True))
#plt.imshow(read_img("inverse_rot_30.jpg"))

In [ ]:
cv2.imwrite("inverse_rot_60.jpg",ICV_rotate(img,60,True))

In [ ]:
cv2.imwrite("inverse_rot_120.jpg",ICV_rotate(img,120,True))

In [ ]:
cv2.imwrite("inverse_rot_-50.jpg",ICV_rotate(img,-50,True))

#### Skewed Images

The second transformation is the skew, which consists of moving the image along the horizontal direction while the vertical side remains fixed.

Two methods have been defined to achieve this.

As with the previous transformation, it must be ensured that no information is lost. Therefore, some borders are added to the input image, but this time, the angle defines how many pixels are going to be added.

In [ ]:
def ICV_add_skew_borders(img,angle):
    #calculate the additional columns needed by getting the length of the triangle that will be formed after the transformation.
    c = len(img[0])/np.tan(angle)
    bordersh = np.zeros_like(img[:,0:int((c)),:])
    img=np.hstack((bordersh,img))
    return img

To skew horizontally an image, next method should be executed with an image and an angle in degrees

In [ ]:
def ICV_skew_horizontally(img,angle):
    #Transformation are applied to the edges, so, our origin angle is 90
    angle=90+angle
    border_angle = 180-angle
    angle=np.deg2rad(angle)
    #add additional borders to the image
    img = ICV_add_skew_borders(img,-angle)
    skewed=np.zeros_like(img)
    #defines the shear matrix
    skew_matrix=[
        [1, 1/np.tan(angle)],
        [1/np.tan(angle), 1]
    ]
    width,height=len(img),len(img[0])
    #starts iterating over all the pixels
    for x in range(width):
        for y in range(height):
            new_pos = np.dot(skew_matrix,np.array([[x],[y]]))
            #computes the new position
            new_x=np.int_(new_pos[0])
            new_y=np.int_(new_pos[1])
            
            if 0 <= new_y and new_y < height:
                skewed[x,new_y]=img[x,y]
    return skewed

In [ ]:
cv2.imwrite("inverse_skew_10.jpg",ICV_skew_horizontally(img,10))

In [ ]:
cv2.imwrite("inverse_skew_40.jpg",ICV_skew_horizontally(img,40))

In [ ]:
cv2.imwrite("inverse_skew_60.jpg",ICV_skew_horizontally(img,60))

Rotation and Skew

In [ ]:
def ICV_rotated_before_skewed_img(img,rotation_angle,skew_angle):
    rotated_img = ICV_rotate(img,rotation_angle,True)
    rotated_and_skewed_img = ICV_skew_horizontally(rotated_img,skew_angle)
    return rotated_and_skewed_img

def ICV_skewed_before_rotated_img(img,rotation_angle,skew_angle):
    skewed_img = ICV_skew_horizontally(img,skew_angle)
    rotated_after_skewed_img = ICV_rotate(skewed_img,rotation_angle,True)
    return rotated_after_skewed_img

In [ ]:
cv2.imwrite("rotated_before_skewed.jpg",ICV_rotated_before_skewed_img(img,20,50))
plt.imshow(read_img("rotated_before_skewed.jpg"))
cv2.imwrite("skewed_before_rotated.jpg",ICV_skewed_before_rotated_img(img,20,50))
plt.imshow(read_img("skewed_before_rotated.jpg"))

## Convolution

In [ ]:
def ICV_load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images


In [ ]:
dataset_BGR= ICV_load_images_from_folder('Dataset/DatasetA')
dataset=[cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY),cv2.COLOR_GRAY2RGB) for img in dataset_BGR]

This is a helper method created to take the same borders from the input image, avoiding losing data after applying a convolution. 


This method is going to be used by ICV_convolution

In [ ]:
def ICV_same_borders(img,w_kernel,h_kernel):
    #Takes the borders of the input images and assigns them to a new matrix with zeros.
    img_borders = np.zeros_like(img)
    #The numbers of columns and rows to add are defined by the size of the kernel
    for i in range(int(w_kernel/2)):
        img_borders[i,:] = img[i,:]
        img_borders[(len(img)-1)-i,:] = img[(len(img)-1)-i,:]
    for i in range(int(h_kernel/2)):
        img_borders[:,i] = img[:,i]
        img_borders[:,(len(img[0])-1)-i] = img[:,(len(img[0])-1)-i]
    return img_borders

Here, the convolution method is defined to apply a given kernel to an input image and return the new image. The kernel should be defined as a matrix

There are two methods defined, one is applied to BGR images and the second can be applied to GRAY images

In [ ]:
def ICV_convolution(img,kernel):
    #Defines the size of the kernel and image
    w_kernel,h_kernel = len(kernel),len(kernel[0])
    w_img,h_img = len(img),len(img[0])
    #assigns the borders from the input to the output image
    new_img = ICV_same_borders(img,w_kernel,h_kernel)
    w_border = int(w_kernel/2)
    h_border = int(h_kernel/2)
    #Start iterating over the image by assigning the new value to the center of each window, for each color
    for i in range(int(w_kernel/2),len(img)-int(w_kernel/2)):
        for j in range(int(h_kernel/2),len(img[0])-int(h_kernel/2)):
            new_img[i,j,0] = np.sum(img[i-w_border:i+w_border+1, j-h_border:j+h_border+1,0] * kernel)
            new_img[i,j,1] = np.sum(img[i-w_border:i+w_border+1, j-h_border:j+h_border+1,1]* kernel)
            new_img[i,j,2] = np.sum(img[i-w_border:i+w_border+1, j-h_border:j+h_border+1,2]* kernel)
    return new_img

#This method has the same logic than ICV_convolution but is created to work with grayscale images
def ICV_convolution_gray(img,kernel):
    w_kernel,h_kernel = len(kernel),len(kernel[0])
    w_img,h_img = len(img),len(img[0])
    new_img = ICV_same_borders(img,w_kernel,h_kernel)
    w_border = int(w_kernel/2)
    h_border = int(h_kernel/2)
    for i in range(int(w_kernel/2),len(img)-int(w_kernel/2)):
        for j in range(int(h_kernel/2),len(img[0])-int(h_kernel/2)):
            new_img[i,j] = np.sum(img[i-w_border:i+w_border+1, j-h_border:j+h_border+1] * kernel)
    return new_img
                                               

Here is the mean kernel that basically takes the mean in a 3x3 matrix and assign its value to each pixel. This could be used to remove noise from the image.

In [ ]:
k2=[[1/9, 1/9 ,1/9],
    [1/9, 1/9 ,1/9],
    [1/9, 1/9 ,1/9]]

This is another way to remove noise from an image, the median kernel, and works by finding the median in an nxn matrix and assigning its value to each pixel

In [ ]:
def ICV_denoise_median_filter(img,kernel):
    #defines the size of the kernel and image
    w_kernel,h_kernel = kernel, kernel
    w_img,h_img = len(img),len(img[0])
    #create a new matrix with the borders of the input image
    new_img = ICV_same_borders(img,w_kernel,h_kernel)
    w_border = int(w_kernel/2)
    h_border = int(h_kernel/2)
    #starts iterating over the image
    for i in range(int(w_kernel/2),len(img)-int(w_kernel/2)):
        for j in range(int(h_kernel/2),len(img[0])-int(h_kernel/2)):
            #for each color it computes the median of an nxn window
            new_img[i,j,0] = np.median(img[i-w_border:i+w_border+1, j-h_border:j+h_border+1,0])
            new_img[i,j,1] = np.median(img[i-w_border:i+w_border+1, j-h_border:j+h_border+1,1])
            new_img[i,j,2] = np.median(img[i-w_border:i+w_border+1, j-h_border:j+h_border+1,2])
    return new_img

In [ ]:
cv2.imwrite("Mean_kernel.jpg",ICV_convolution(dataset_BGR[0],k2))
cv2.imwrite("Median_kernel.jpg",ICV_denoise_median_filter(dataset_BGR[0],3))

Here both kernels A and B are defined and normalized (In the case of B it is not possible to normalize it because the sum gives 0).


After that, each one is applied to the image by using ICV_convolution method.

In [ ]:
k_A=[[1, 2 ,1],
[2, 4 ,2],
[1, 2 ,1]]

k_A = k_A/np.sum(k_A)

k_B = [[0, 1 ,0],
[1, -4 ,1],
[0, 1 ,0]]

img_k_a = ICV_convolution(dataset_BGR[0],k_A)
img_k_b = ICV_convolution(dataset_BGR[0],k_B)

cv2.imwrite("kernel_A.jpg",ICV_convolution(dataset_BGR[0],k_A))
cv2.imwrite("kernel_B.jpg",ICV_convolution(dataset_BGR[0],k_B))

Here, I'm going to test what I get if these kernels are combined.

In [ ]:
#A followed by A
cv2.imwrite("kernel_A_followed_by_A.jpg",ICV_convolution(img_k_a,k_A))
#A followed by B
cv2.imwrite("kernel_A_followed_by_B.jpg",ICV_convolution(img_k_a,k_B))
#B followed by A
cv2.imwrite("kernel_B_followed_by_A.jpg",ICV_convolution(img_k_b,k_A))

## Video segmentation

In this section our input is going to be a video, so the first step is to open it and save each frame in an array

In [ ]:
def ICV_get_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)

    cap.release()
    return frames

This is a helper method that is going to be used by some of the methods to create an histogram of a given matrix and a number of bins.

In [ ]:
def ICV_histogram(m,bins):
    hist = [np.sum(np.where(m==i,1,0)) for i in range(0,bins)]
    return hist

Here, ICV_img_histogram was implemented to create an histogram for each color of a given BGR image and plot it

In [ ]:
def ICV_img_histogram(img,i):
    R = img[:,:,2]
    G = img[:,:,1]
    B = img[:,:,0]
    #computes the histogram for each color
    R_hist=ICV_histogram(R,256)
    G_hist=ICV_histogram(G,256)
    B_hist=ICV_histogram(B,256)
    bins = range(len(R_hist))
    plt.figure()
    #Plots each histogram in a single plot
    plt.plot(bins,R_hist,label="R",color='r')
    plt.plot(bins,G_hist, label="G",color='g')
    plt.plot(bins,B_hist,label="B",color='b')
    plt.savefig("video_hist/fig_"+str(i)+".jpg")
    plt.close()

Before executing the code, it's important to make sure all directories exists and if not, create them. This can be done by executing the next cell. 

In [ ]:
os.makedirs('video_hist', exist_ok=True)
os.makedirs('video_hist_intersection', exist_ok=True)
os.makedirs('video_hist_intersection_norm', exist_ok=True)
os.makedirs('texture_classification', exist_ok=True)
os.makedirs('texture_classification/windows', exist_ok=True)
os.makedirs('texture_classification/windows_lbp', exist_ok=True)

Here I'm going to iterate between all the frames of the video and plot their histogram.

In [ ]:
frames = ICV_get_frames('Dataset/DatasetB/DatasetB.avi')
for i,frame in enumerate(frames):
    ICV_img_histogram(frame,i)
    cv2.imwrite("video_hist/input_fig_"+str(i)+".jpg",frame)

This method can be used to normalize each color histogram and plot the result

In [ ]:
def ICV_normalize(hist):
    #get the norm values by dividing each value by the total for each color
    hist[0] = np.array(hist[0])/sum(hist[0])
    hist[1] = np.array(hist[1])/sum(hist[1])
    hist[2] = np.array(hist[2])/sum(hist[2])
    return hist
def ICV_normalize_gray(hist):
    #get the norm values by dividing each value by the total for each color
    hist = np.array(hist)/sum(hist)
    return hist

This method finds the intersection between 2 histograms by taking the min value for each bin and saves the plot

In [ ]:
def ICV_hist_intersection(hist_a,hist_b,bins,actual):
    bins=range(bins-1)
    #computes the intersection by calculation the min between the histograms in each bin
    hist = [[min(hist_a[j][i],hist_b[j][i]) for i in range(len(hist_a[0]))]for j in range(len(hist_a))]
    #plots the intersection and saves the image
    plt.figure()
    plt.plot(bins,hist[0],label="B",color='b')
    plt.plot(bins,hist[1],label="G",color='g')
    plt.plot(bins,hist[2],label="R",color='r')
    plt.savefig("video_hist_intersection/fig_"+str(actual)+"_"+str(actual+1)+".jpg")
    plt.close()
    #Normalizes the histogram and saves the plot
    norm = ICV_normalize(hist)
    plt.figure()
    plt.plot(bins,norm[0],label="B",color='b')
    plt.plot(bins,norm[1],label="G",color='g')
    plt.plot(bins,norm[2],label="R",color='r')
    plt.savefig("video_hist_intersection_norm/fig_"+str(actual)+"_"+str(actual+1)+".jpg")
    
    plt.close()

Now, I'm going to iterate over each frame of the video, to calculate the individual histogram, the intersection between the actual and the next histogram, and the normalized intersection histogram.

In [ ]:
for i in range(len(frames)-1):
    #defines the frames to compare
    actual = frames[i]
    next = frames[i+1]
    #Gets each single histogram
    hist_a = [ICV_histogram(actual[:,:,i],255) for i in range(3)]
    hist_b = [ICV_histogram(next[:,:,i],255) for i in range(3)]
    #Computes their intersection
    ICV_hist_intersection(hist_a,hist_b,256,i)

### 4. Texture clasification

It's time to apply the definitions and processes of texture classification to finally define to which class belongs each image from the dataset A. 

In [ ]:
from scipy.spatial.distance import euclidean

def ICV_load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images[filename] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return images

images = ICV_load_images_from_folder('Dataset/DatasetA')


**These are the helper methods that are going to be used by our classificator**

Next method is going to iterate over the pixels of the window to transform each value into binary depending if it is bigger or smaller than the center value. After getting a binary matrix, transforms it into a binary number and finally computes to get the corresponding decimal and assign it to the center.

In [ ]:
def ICV_LBP_values(window,count):
    size=len(window)
    lbp_values = []
    global_descriptor = []
    pos_window = np.zeros_like(window)
    for i in range(1,size-1):
        for j in range(1,size-1):
            #Extract the sub_img of 3x3 to get the binary and finally the nuew value of the position
            sub_img = window[i-1:i+2, j-1:j+2]
            #get each value of the subimage clockwise and save them in a list
            clock_wise= [[0,0],[0,1],[0,2],[1,2],[2,2],[2,1],[2,0],[1,0]]
            sub_img = [sub_img[pos[0],pos[1]] for pos in clock_wise]
            #Add 1 if value is bigger than the center, 0 else.
            binary = [1 if bit > window[i,j] else 0 for bit in sub_img]
            #calculate the binary and assign it to the window
            decimal = sum([b<<l for l, b in enumerate(binary[::-1])])
            pos_window[i,j]=decimal
            lbp_values.append(decimal)
    normalized_hist = ICV_LBP_window_hist(pos_window[1:-1,1:-1],count)
    return pos_window, normalized_hist


This method will be used to create and save ICV_histogram of a given LBP window

In [ ]:
def ICV_LBP_window_hist(window,count):
    #Computes the histogram of the window and normalizes it
    lbp_hist=ICV_histogram(window,256)
    lbp_hist = ICV_normalize_gray(lbp_hist)
    #plots the histogram and saves the image
    plt.plot(range(256),lbp_hist,label="R",color='r')
    plt.savefig("texture_classification/fig_"+str(count)+".jpg")
    plt.close()
    return lbp_hist

In [ ]:
def ICV_plot_lbp_hist(hist,name):
    #plots a given hist and saves the image
    bins = range(len(hist))
    plt.plot(bins,hist,label="R",color='r')
    plt.savefig("texture_classification/fig_"+str(name)+".jpg")
    plt.close()

By calling this method, the intersection of all histograms can be done to finally get the global descriptor. It the already intersected histograms and the new histogram to intersect. Finally returns the intersection between them.

In [ ]:
def ICV_lbp_intersection(global_descriptor,normalized_hist):
    #If it is the first iteration, assigns the histogram of the frame 0, else, computes the min value between the global descriptor and the new frame histfor each bin
    if len(global_descriptor) > 1:
        hist = [min(global_descriptor[i],normalized_hist[i]) for i in range(len(normalized_hist))]
    else:
        hist=normalized_hist
        
    return hist

This basically returns how far one histogram is from another by calculating their Euclidean distance. This method will be used to compare global histograms and define whether they both belong to the same class.

In [ ]:
def ICV_lbp_comparator_euclidean(lbp_hist,lbp_hist2):
    return euclidean(lbp_hist,lbp_hist2)

**This is the main method that could be used to get the global histogram of a given image and the image divided in windows of w_size size**

In [ ]:
def ICV_LBP_windows(source_img,w_size,name):
    count=0
    #creates a matrix with the same size of the image
    source = np.zeros_like(source_img)
    global_descriptor_intersec = []
    #Starts iterating over the image by each window size
    for i in range(0,len(source_img),w_size):
        for j in range(0,len(source_img[0]),w_size):
            #gets the window
            w = source_img[j:j+w_size,i:i+w_size]
            #Check if the window has the required size
            if len(w)==w_size and len(w[0])==w_size:
                #Computes the LBP descriptor and updates the output image
                source[j:j+w_size,i:i+w_size],normalized_hist = ICV_LBP_values(w,count)
                #computes the intersection between the global and the actual window
                global_descriptor_intersec = ICV_lbp_intersection(global_descriptor_intersec,normalized_hist)
                count+=1 
    ICV_plot_lbp_hist(global_descriptor_intersec,"global_intersect"+name)
    return source,global_descriptor_intersec

Now, by using this class with the name of the image (ubicated in Dataset/DatasetA) is going to return if it is a car or a face by calculating the euclidean distance from face_1 and car_1

In [ ]:
class ICV_lbp_classifier():
    def __init__(self,window_size):
        self.window_size = window_size
        #computes the global descriptor for the base face image
        self.face_1_lbp,self.base_intersec_face_hist = ICV_LBP_windows(images["face-1.jpg"],window_size,"face____1")
    
    def is_face(self,imgs_name):
        self.faces,self.non_faces = [],[]
        #Iterates over every image
        for img_name in imgs_name:
            #Computes its LBP
            im_lbp,intersec_hist = ICV_LBP_windows(images[img_name],self.window_size,img_name)
            #Calculates the distance to the base image
            distance_face = ICV_lbp_comparator_euclidean(intersec_hist,self.base_intersec_face_hist)
            #If the distance is less than 0.03 the image is a face
            if distance_face < 0.03:
                self.faces.append(img_name)
            else:
                self.non_faces.append(img_name)

In [ ]:
lbp_30 = ICV_lbp_classifier(30)
lbp_30.is_face(images.keys())
print("Faces : {}  Non Faces: {}".format(lbp_30.faces,lbp_30.non_faces))

In [ ]:
lbp_200 = ICV_lbp_classifier(100)
lbp_200.is_face(images.keys())
print("Faces : {}  Non Faces: {}".format(lbp_200.faces,lbp_200.non_faces))

In [ ]:
lbp_20 = ICV_lbp_classifier(20)
lbp_20.is_face(images.keys())
print("Faces : {}  Non Faces: {}".format(lbp_20.faces,lbp_20.non_faces))

### Object counting

In this case, to apply the object counting theory learned in class, we are going to use and AVI file named DatasetC, to extract all its frames and compare them.

In [ ]:
#Creating the folder if it doesn't exist.
os.makedirs('object_counting', exist_ok=True)

In [ ]:
#Extract each frame and convert them to gray scale
frames_to_count = ICV_get_frames('Dataset/DatasetC.avi')
frames_to_count =[cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in frames_to_count]

**Helper method:**


This method, give a given image, assigns 255 to every value bigger than a given treshold, else 0

In [ ]:
def ICV_treshold(s_img,tresh):
    treshold_img = np.zeros_like(s_img)
    #Iterates over the image
    for i in range(len(s_img)):
        for j in range(len(s_img[0])):
            #Assigns 255 to any pixel greater than the threshold
            if s_img[i,j] > tresh:
                treshold_img[i,j] = 255
            
    return treshold_img    

def ICV_abs_difference(frame_1,frame_2):
	matrix = [[abs(int(frame_1[j,i])-int(frame_2[j,i])) for i in range(len(frame_1[0]))]for j in range(len(frame_1))]
	return matrix


Now to calculate the pixel-by-pixel frame differencing and get the tresholded image, the next method sould be executed with a list of frames and an optional flag to compare all the frames with an specific position (the index of the frame should be passed)

In [ ]:
def ICV_frame_differencing(frames_to_diff,frame_base=-1,):
    #Get the number of frames to be compared
    n_frames = len(frames_to_diff)
    #in case frame_base is different to -1, every frame is compared to the specified position by getting the absolute
    #difference and each value is compared to the threshold defined, to finally get the image with the detected objects.
    if frame_base != -1 and frame_base <= n_frames:
        for i in range(n_frames):
            frame_1,frame_2 = frames_to_diff[frame_base],frames_to_diff[i]
            abs_difference = [[abs(int(frame_1[j,i])-int(frame_2[j,i])) for i in range(len(frame_1[0]))]for j in range(len(frame_1))]
            abs_difference= np.array(abs_difference,dtype=np.uint8)
            treshold_img = ICV_treshold(np.array(abs_difference),25)
            cv2.imwrite("object_counting/_specific_frame"+str(frame_base)+"_and_"+str(i)+".jpg",cv2.cvtColor(treshold_img, cv2.COLOR_GRAY2RGB))
    else:
        #else, frame_base is -1, every frame is compared to next frame
        for i in range(n_frames-1):
            frame_1,frame_2 = frames_to_diff[i],frames_to_diff[i+1]
            abs_difference = [[abs(int(frame_1[j,i])-int(frame_2[j,i])) for i in range(len(frame_1[0]))]for j in range(len(frame_1))]
            abs_difference= np.array(abs_difference,dtype=np.uint8)
            treshold_img = ICV_treshold(np.array(abs_difference),25)
            cv2.imwrite("object_counting/frames_"+str(i)+"_and_"+str(i+1)+".jpg",cv2.cvtColor(treshold_img, cv2.COLOR_GRAY2RGB))
    

This line generates all the images with the treshold of the difference between the frame 0 and each frame

In [ ]:
ICV_frame_differencing(frames_to_count,0)

This line generates all the images with the treshold of the difference between all consecutived frames

In [ ]:
ICV_frame_differencing(frames_to_count)

Next method could be used to get the background of a given list of frames

In [ ]:
def ICV_get_back(frames):
    #Assigns the first frame
    bg=frames[0]
    n_frames = len(frames)
    w = 0.01
    #starts iterating over each frame
    for i in range(1,n_frames):
        frame_1 = frames[i]
        #get the difference between consecutive images
        abs_difference_matrix = np.array(ICV_abs_difference(bg,frame_1))
        treshold_matrix = ICV_treshold(np.array(abs_difference_matrix,dtype=np.uint8),25)
        #Computed the weighted average given more weight to the background
        bg = (bg* (1-(0.02)))+ (abs_difference_matrix*(0.02))
        w += 0.01
   
    bg=bg/w
    return bg
cv2.imwrite("object_counting/background.jpg",np.matrix(ICV_get_back(frames_to_count),dtype=np.uint8))
background = ICV_get_back(frames_to_count)

Next method will count the number of objects of a given threshold matrix

In [ ]:
def ICV_object_count(tresh_img):
    #objects and their pixels
    objects = dict()

    for i in range(tresh_img.shape[0]):
        for j in range(tresh_img.shape[1]):
            #check if the pixel is white
            if tresh_img[i, j] == 255:
                is_new = True
                detected_objs = []
                #If there are already found objects finds how near is from them
                for obj in objects.keys():
                    is_new = ICV_is_near(objects[obj],i,j)
                    if not is_new:
                        detected_objs.append(obj)
                #If the pixel is not near to any found object, add it as a new object to the list            
                if is_new == True:
                    objects[len(objects)+1] = [(i, j)]
                #If it belongs to any object adds it to the list 
                else:
                    if len(detected_objs) > 1:
                        for object_d in detected_objs[1:]:
                            objects[detected_objs[0]].extend(objects[object_d])
                            objects.pop(object_d)
                    else:
                        if (i,j) not in objects[detected_objs[0]]:
                            objects[detected_objs[0]].append((i,j))

    count = sum([1 for key in objects.keys() if len(objects[key]) > 50])
    return count

In [ ]:
def ICV_is_near(objects_obj,i,j):
    is_newv=True
    for r,c in objects_obj:
            #if the pixel is near to an object by 4 or less pixels, it is not consider as a new object
        if abs(i - r) <= 3 and abs(j - c) <= 3:
            is_newv = False
            break
    return is_newv

In [ ]:
image = cv2.imread("object_counting/_specific_frame0_and_1.jpg")
image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

Here, I'm going to iterate over each frame to get the number of moving objects in all of them

In [ ]:
def ICV_objects_plot(frames,background_im):
    count = np.zeros(len(frames))
    for i in range(len(frames)):
            frame_2 = frames[i]
        #Computes he difference and the threshold of 2 consecutive frames
            abs_difference = [[abs(int(background_im[j,i])-int(frame_2[j,i])) for i in range(len(frame_2[0]))]for j in range(len(frame_2))]
            abs_difference= np.array(abs_difference,dtype=np.uint8)
            treshold_img = ICV_treshold(np.array(abs_difference),25)
            cv2.imwrite("object_counting/count"+str(i)+".jpg",cv2.cvtColor(treshold_img, cv2.COLOR_GRAY2RGB))
        #By using the defined method, count the number of objects and adds it to the count list
            count[i] = ICV_object_count(treshold_img)                       
    return count

Here im going to execute the above code

In [ ]:
objects_count = ICV_objects_plot(frames_to_count,frames_to_count[0])

In [ ]:
plt.bar(range(140),objects_count)
plt.savefig("count_object_graph.jpg")
plt.close()